In [16]:
 

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib




In [17]:
# Charger les données
df = pd.read_csv("Walmart_Sales.csv")





In [4]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [18]:
# Prétraitement simple
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)



In [19]:
df['Month'] = df['Date'].dt.month
df['Week'] = df['Date'].dt.isocalendar().week
df['Holiday_Flag'] = df['Holiday_Flag'].astype(int)
df.drop(columns=['Date'], inplace=True)

In [20]:
# Définir features et cible
X = df.drop("Weekly_Sales", axis=1)
y = df["Weekly_Sales"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [21]:
# Modèle
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



RandomForestRegressor(random_state=42)

In [22]:
# Évaluer
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE: {rmse}")



RMSE: 114531.36738523633


In [23]:
# Sauvegarder le modèle
joblib.dump(model, "sales_model.pkl")

['sales_model.pkl']

In [24]:
def preprocess_input(data: SalesInput):
    df = pd.DataFrame([data.dict()])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week
    df['IsHoliday'] = df['IsHoliday'].astype(int)
    
    # Ajouter Holiday_Flag comme une colonne calculée
    df['Holiday_Flag'] = df['IsHoliday']  # Peut-être que vous deviez avoir un calcul différent
    
    # Supprimer la colonne 'Date' qui n'est plus nécessaire
    df.drop(columns=['Date'], inplace=True)
    
    return df


In [25]:
from pydantic import BaseModel
import pandas as pd
import joblib

# Définir la classe SalesInput
class SalesInput(BaseModel):
    Store: int
    Dept: int
    Temperature: float
    Fuel_Price: float
    CPI: float
    Unemployment: float
    IsHoliday: bool
    Date: str

# Charger le modèle
model = joblib.load("sales_model.pkl")

 

# Données de test
test_data = {
    'Store': 1,
    'Dept': 1,
    'Temperature': 72.5,
    'Fuel_Price': 3.5,
    'CPI': 211.3,
    'Unemployment': 8.5,
    'IsHoliday': True,
    'Date': '2023-12-25'
}



In [27]:
from pydantic import BaseModel
import pandas as pd
import joblib

# Définir la classe SalesInput
class SalesInput(BaseModel):
    Store: int
    Dept: int
    Temperature: float
    Fuel_Price: float
    CPI: float
    Unemployment: float
    IsHoliday: bool
    Date: str

# Charger le modèle
model = joblib.load("sales_model.pkl")

# Fonction de prétraitement des données
def preprocess_input(data: SalesInput):
    df = pd.DataFrame([data.dict()])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week
    df['IsHoliday'] = df['IsHoliday'].astype(int)
    
    # Ajouter Holiday_Flag si nécessaire pour correspondre au modèle
    df['Holiday_Flag'] = df['IsHoliday']  # Cette ligne dépend de ce que votre modèle attend

    # Assurez-vous que les colonnes présentes lors de l'entraînement sont celles que vous envoyez
    expected_columns = ['Store', 'Dept', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Week', 'Holiday_Flag']
    
    # Ajuster les données pour correspondre aux colonnes attendues
    df = df[expected_columns]
    
    # Supprimer la colonne 'Date' qui n'est plus nécessaire
    df.drop(columns=['Date'], inplace=True)
    
    return df

# Données de test
test_data = {
    'Store': 1,
    'Dept': 1,
    'Temperature': 72.5,
    'Fuel_Price': 3.5,
    'CPI': 211.3,
    'Unemployment': 8.5,
    'IsHoliday': True,
    'Date': '2023-12-25'
}



In [32]:
def preprocess_input(data: SalesInput):
    # Pydantic v2 : utiliser model_dump() au lieu de dict()
    df = pd.DataFrame([data.model_dump()])
    
    # Convertir la date
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week

    # Créer la colonne attendue par le modèle
    df['Holiday_Flag'] = int(data.IsHoliday)

    # Garder uniquement les colonnes attendues, dans le bon ordre
    final_columns = ['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Week']
    df = df[final_columns]

    return df


In [33]:
# Créer une instance de SalesInput avec les données de test
test_input = SalesInput(**test_data)



In [30]:
print(X_train.columns.tolist())


['Store', 'Holiday_Flag', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Month', 'Week']


In [34]:
# Prétraiter les données
processed_input = preprocess_input(test_input)

# Faire la prédiction
prediction = model.predict(processed_input)
print(f"Test prediction: {prediction}")


Test prediction: [1739867.0712]
